<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Build and Save a Scikit-Learn model to WSL Repository</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
</table>

This notebook walks you through these steps:
 
- Access the data
- Cleanse data for analysis
- Explore data
- Build a classification model
- Save the model in the ML repository with associated meta data


### Step 1: Download Data as a Pandas Dataframe from Local file storage

Read the three Mortgage files from Local file storage - Insert them as PANDAS Dataframes

In [ ]:
# Load the pandas libraries

import os, pandas as pd

In [ ]:
# Import the CUSTOMER data 

df_data_1 = pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/MortgageCustomer.csv')
df_data_1.head()

In [ ]:
# Imprt the PROPERTY data

df_data_2 = pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/Property.csv')
df_data_2.head()

In [ ]:
# Import the DEFAULT data

df_data_3 = pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/Default.csv')
df_data_3.head()



#### Rename dataframes and drop any unwanted fields

In [ ]:
# Rename the default dataframe names to something more meaningful and drop any 

customer = df_data_1
property = df_data_2
default = df_data_3

In [ ]:
# Check that the dataframes have the desired columns

print ("Customer dataframe:")
print (list(customer))
print ("")
print ("Property dataframe:")
print (list(property))
print ("")
print ("Default dataframe:")
print (list(default))

### Step 2: Merge Files

In [ ]:
merged = pd.merge(pd.merge(customer, property, on='ID'),default,on='ID')
merged.head(3)

### Step 3: Simple Data Preparation - Rename some columns and ensure correct data types 
This step is to remove spaces from columns names

In [ ]:
merged = merged.rename(index=str, columns={"Yrs at Current Address":"YearCurrentAddress", "Yrs with Current Employer":"YearsCurrentEmployer",\
                      "Number of Cards":"NumberOfCards","Creditcard Debt":"CCDebt","Loan Amount":"LoanAmount"})
merged.head(3)

Check data types and re-cast numeric fields to **Integers**

In [ ]:
merged.dtypes

In [ ]:
merged[['ID','Income','YearCurrentAddress','YearsCurrentEmployer','NumberOfCards','CCDebt','Loans','LoanAmount']] = \
merged[['ID','Income','YearCurrentAddress','YearsCurrentEmployer','NumberOfCards','CCDebt','Loans','LoanAmount']].astype(int)
merged.dtypes

### Step 4: Data Exploration

1) Obtain some data shape summaries in terms of number of fields and records <br>
2) Perform some exploratory analysis of distributions, scatterplots using two different graphics packages


In [ ]:
print ("There are " + str(merged.shape[0]) + " records and " + str(merged.shape[1]) + " fields in the dataset.")

In [ ]:
import brunel
%brunel data('merged') bar x(Residence) y(Income) mean(CCDebt) color(MortgageDefault) stack tooltip(Income) \
| x(YearCurrentAddress) y(YearsCurrentEmployer) point color(MortgageDefault) tooltip(YearCurrentAddress, YearsCurrentEmployer) \
:: width=1100, height=400 

In [ ]:
from pixiedust.display import *
display(merged)

### Step 5: Build the XGBoost model

In recent years, ensemble learning models took the lead and became popular among machine learning practitioners.

Ensemble learning model employs multiple machine learning algorithms to overcome the potential weaknesses of a single model. For example, if you are going to pick a destination for your next vacation, you probably ask your family and friends, read reviews and blog posts. Based on all the information you have gathered, you make your final decision.

This phenomenon is referred as the Wisdom of Crowds (WOC) in social sciences and it states that averaging the answers (prediction or probability) of a group will often result better than the answer of one of its members. The idea is that the collective knowledge of diverse and independent individuals will exceed the knowledge of any one of those individuals, helping to eliminate the noise.

XGBoost is an open source library for ensemble based algorithms. It can be used for classification, regression and ranking type of problems. XGBoost supports multiple languages, such as C++, Python, R, and Java. 

The Python library of XGBoost supports the following API interfaces to train and predict a model, also referred to as a `Booster`: 
- XGBoost's native APIs pertaining to the `xgboost` package, such as `xgboost.train()` or `xgboost.Booster`
- Scikit-Learn based Wrapper APIs: `xgboost.sklearn.XGBClassifier` and `xgboost.sklearn.XGBRegressor`

Details about using the scikit-learn based Wrapper APIs to create and predict an XGBoost model is explained in the the [Classify tumors with machine learning](https://apsportal.ibm.com/exchange/public/entry/view/ac820b22cc976f5cf6487260f4c8d9c8) notebook.

In this section you will learn how to train and test an XGBoost model using XGBoost's native python APIs. 

First, you must import the required libraries.

In [ ]:
import xgboost as xgb

import pandas as pd

from sklearn import cross_validation
from sklearn.cross_validation import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

from matplotlib import pyplot
import pprint
%matplotlib inline
import numpy as np


### 5.1: Prepare data

In this section, clean and transform the data in the Pandas data frame into the data that can be given as input for training the model. 

In [ ]:
print("List of features with their corresponding count of null values : ")
print("---------------------------------------------------------------- ")
print(merged.isnull().sum())

#### 5.1.2: Prepare the target data and feature columns

In this section, transform the existing data frame to derive the target data that contains the prediction value for the corresponding sample data. 

The goal of the model here is to predict whether a mortgage customer will default on their loan. The column Mortgage Default is currently a VarChar with values YES/NO and therefore it needs to be converted to a numeric

In [ ]:
# Convert the VarChar columns to numeric columns

le = LabelEncoder()
merged.loc[:,'MortgageDefault']= le.fit_transform(merged.loc[:,'MortgageDefault'])
merged.head()

In [ ]:
# Check the values for MortgageDefault

merged.groupby(['MortgageDefault']).size()

In [ ]:
# Only need to run this once within your notebook

!pip install sklearn-pandas

In [ ]:
!pip install mxnet --no-deps

In [ ]:
# Apply the LabelEncoder to encode the input features in numeric form where applicable
from sklearn_pandas import DataFrameMapper

mapper = DataFrameMapper([
     ('Income', None),
     ('AppliedOnline', LabelEncoder()),
     ('Residence', LabelEncoder()),
     ('YearCurrentAddress',None),
     ('YearsCurrentEmployer',None),
     ('NumberOfCards',None),
     ('CCDebt',None),
     ('Loans',None),
     ('LoanAmount',None),
     ('SalePrice',None),
     ('Location',None)
    ])

In [ ]:
# Drop the Varchar fields from the analysis
feature_cols = ['Income','YearCurrentAddress','YearsCurrentEmployer','NumberOfCards','CCDebt','Loans','LoanAmount','SalePrice','Location']
features_df = merged[feature_cols]
features_df.head()

#### 5.1.3: Split the data set for training and testing

As the target and feature columns has been defined, you can now split the data set into two sets that will be used for training the model and for testing the trained model. 

In [ ]:
# define the label (Target Field) and features
# split the data to training and testing set
y = np.float32(merged.MortgageDefault)
x = features_df
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

#### 5.1.4: Construct DMatix objects 

DMatrix is the data interface provided by the XGBoost library. The training data and test data are converted as DMatrix objects to perform training and to make predictions. The DMatrix objects can be created from various data formats, such as Numpy arrays, Pandas data frames, or a Scipy sparse array. For more information about the DMatrix interface, see [Python Package Introduction](http://xgboost.readthedocs.io/en/latest/python/python_intro.html).

In [ ]:
# Prepare the **DMatrix** objects for training and testing based on the training and test data that was split above
dm_train = xgb.DMatrix(x_train, label=y_train)
dm_test = xgb.DMatrix(x_test)

### 5.2: Create XGBoost model (Booster)

In [ ]:
# Set the parameters of the Booster that we are about to create and train.

param = {'objective':'multi:softmax', 'max_depth':2, 'eta':0.8, 'num_class': 2, 'eval_metric': 'auc', 'silent':1 }

In [ ]:
# Create a Booster by using the training data set, which is in the form of a DMatrix object.

xgb_model = xgb.train(param, dm_train)

### Step 6: Model Evaluation 

In [ ]:
# call xgb_model.predict() on your matrix test data to make a set of test predictions which are written to series y_predict

y_predict = xgb_model.predict(dm_test)
print(y_predict)


In [ ]:
# Look at the accuracy of the XGBoost model
accuracy = accuracy_score(y_test, y_predict)
print("Accuracy: " +  str(accuracy))


### Step 7: Understanding the XGBoost Model

#### 7.1: View the scored "Test" data

In [ ]:
#Reset the index on the x_train data so that the join will match record by record and not require a key
x_test.reset_index(drop=True, inplace=True)

#Write the Actual and Predicted Mortgage Default values in to dataframes 
y_test_df = pd.DataFrame(y_test,columns=['MortgageDefault'])
y_pred_df = pd.DataFrame(y_predict,columns=['Pred Default'])

# Combine the three dataframes by index value rather than key field
scored_df = pd.concat([x_test, y_test_df, y_pred_df], axis=1)
scored_df.head()


#### 7.2: Model Interpretation

To understand the model better, XGBoost provides APIs that you can use to get insights about the trees used for training the model and the importance of the features in constructing the Booster.

In [ ]:
# Import plot / graph libraries required for model interpretation
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Plot the feature importance
xgb.plot_importance(xgb_model)

### Step 8: Save Model in ML repository


In [ ]:
from dsx_ml.ml import save

model_name = "Predict Mortgage Default XGBoost"
save(name = model_name,
     model = xgb_model,
     algorithm_type = 'Classification',
     test_data = x_test)